In [30]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn.neural_network as sknn


%matplotlib inline
sns.set()

In [31]:
DATA_DIR = Path.cwd().parent / 'gaec_altlabs'

In [32]:
train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='sequence_id')
train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='sequence_id')
test_values = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='sequence_id')

In [33]:
test_values.head()

,sequence,bacterial_resistance_ampicillin,bacterial_resistance_chloramphenicol,bacterial_resistance_kanamycin,bacterial_resistance_other,bacterial_resistance_spectinomycin,copy_number_high_copy,copy_number_low_copy,copy_number_unknown,growth_strain_ccdb_survival,...,species_budding_yeast,species_fly,species_human,species_mouse,species_mustard_weed,species_nematode,species_other,species_rat,species_synthetic,species_zebrafish
sequence_id,,,,,,,,,,,,,,,,,,,,,
E0VFT,AGATCTATACATTGAATCAATATTGGCAATTAGCCATATTAGTCAT...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
TTRK5,GCGCGCGTTGACATTGATTATTGACTAGTTATTAATAGTAATCAAT...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2Z7FZ,GCTTAAGCGGTCGACGGATCGGGAGATCTCCCGATCCCCTATGGTG...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VJI6E,ATGATGATGATGTCCCTGAACAGCAAGCAGGCGTTTAGCATGCCGC...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
721FI,GGTACCGAGCTCTTACGCGTGCTAGCCATACTATCAGCCACTTGTG...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
train_labels.head()

,00Q4V31T,012VT4JK,028IO5W2,03GRNN7N,03Y3W51H,09MQV1TY,0A4AHRCT,0A9M05NC,0B9GCUVV,0CL7QVG8,...,ZQNGGY33,ZSHS4VJZ,ZT1IP3T6,ZU6860XU,ZU6TVFFU,ZU75P59K,ZUI6TDWV,ZWFD8OHC,ZX06ZDZN,ZZJVE4HO
sequence_id,,,,,,,,,,,,,,,,,,,,,
9ZIMC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5SAQC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E7QRO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CT5FP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7PTD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# get the column with the max value in each row
lab_ids = pd.DataFrame(train_labels.idxmax(axis=1), columns=['lab_id'])

In [36]:
lab_ids.tail()

,lab_id
sequence_id,
BOQSD,3EZXYI3U
5XVVU,3EZXYI3U
CVGHF,3EZXYI3U
ZVT1A,3EZXYI3U
U5MR3,3EZXYI3U


In [37]:
# Sort lab ids by prevalence
(lab_ids['lab_id'].value_counts(normalize=True)
        .sort_values(ascending=False)).head()

I7FXTVDP    0.131488
RKJHZGDQ    0.043353
GTVTUGVY    0.042401
A18S09P2    0.016884
Q2K8NHZY    0.015440
Name: lab_id, dtype: float64

In [38]:
bases = set(''.join(train_values.sequence.values))

In [39]:
from itertools import permutations

n = 4
subsequences = [''.join(permutation) for permutation in permutations(bases, r=n)]

In [40]:
print(f"Number of subsequences: {len(subsequences)}")
subsequences[:10]

Number of subsequences: 120


['GCTA',
 'GCTN',
 'GCAT',
 'GCAN',
 'GCNT',
 'GCNA',
 'GTCA',
 'GTCN',
 'GTAC',
 'GTAN']

In [41]:
print(f"Number of subsequences: {len(subsequences)}")
subsequences[:10]

Number of subsequences: 120


['GCTA',
 'GCTN',
 'GCAT',
 'GCAN',
 'GCNT',
 'GCNA',
 'GTCA',
 'GTCN',
 'GTAC',
 'GTAN']

In [42]:
"gattattattaca".count("atta")

2

In [43]:
def get_ngram_features(data, subsequences):
    """Generates counts for each subsequence.

    Args:
        data (DataFrame): The data you want to create features from. Must include a "sequence" column.
        subsequences (list): A list of subsequences to count.

    Returns:
        DataFrame: A DataFrame with one column for each subsequence.
    """
    features = pd.DataFrame(index=data.index)
    for subseq in subsequences:
        features[subseq] = data.sequence.str.count(subseq)
    return features

In [47]:
# Calculate n-gram features on our training set
ngram_features = get_ngram_features(train_values, subsequences)
ngram_features.head()

,GCTA,GCTN,GCAT,GCAN,GCNT,GCNA,GTCA,GTCN,GTAC,GTAN,...,NTCG,NTCA,NTAG,NTAC,NAGC,NAGT,NACG,NACT,NATG,NATC
sequence_id,,,,,,,,,,,,,,,,,,,,,
9ZIMC,27,0,20,0,0,0,28,0,15,0,...,0,0,0,0,0,0,0,0,0,0
5SAQC,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
E7QRO,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
CT5FP,3,0,3,0,0,0,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7PTD8,2,0,3,0,0,0,7,0,4,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
ngram_features.shape

(63017, 120)

In [48]:
all_features = ngram_features.join(train_values.drop('sequence', axis=1))

In [49]:
all_features.head()

,GCTA,GCTN,GCAT,GCAN,GCNT,GCNA,GTCA,GTCN,GTAC,GTAN,...,species_budding_yeast,species_fly,species_human,species_mouse,species_mustard_weed,species_nematode,species_other,species_rat,species_synthetic,species_zebrafish
sequence_id,,,,,,,,,,,,,,,,,,,,,
9ZIMC,27,0,20,0,0,0,28,0,15,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5SAQC,1,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
E7QRO,1,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
CT5FP,3,0,3,0,0,0,3,0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7PTD8,2,0,3,0,0,0,7,0,4,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
def top10_accuracy_scorer(estimator, X, y):
    """A custom scorer that evaluates a model on whether the correct label is in 
    the top 10 most probable predictions.

    Args:
        estimator (sklearn estimator): The sklearn model that should be evaluated.
        X (numpy array): The validation data.
        y (numpy array): The ground truth labels.

    Returns:
        float: Accuracy of the model as defined by the proportion of predictions
               in which the correct label was in the top 10. Higher is better.
    """
    # predict the probabilities across all possible labels for rows in our training set
    probas = estimator.predict_proba(X)
    
    # get the indices for top 10 predictions for each row; these are the last ten in each row
    # Note: We use argpartition, which is O(n), vs argsort, which uses the quicksort algorithm 
    # by default and is O(n^2) in the worst case. We can do this because we only need the top ten
    # partitioned, not in sorted order.
    # Documentation: https://numpy.org/doc/1.18/reference/generated/numpy.argpartition.html
    top10_idx = np.argpartition(probas, -10, axis=1)[:, -10:]
    
    # index into the classes list using the top ten indices to get the class names
    top10_preds = estimator.classes_[top10_idx]

    # check if y-true is in top 10 for each set of predictions
    mask = top10_preds == y.reshape((y.size, 1))
    
    # take the mean
    top_10_accuracy = mask.any(axis=1).mean()
 
    return top_10_accuracy

In [51]:
from sklearn.ensemble import RandomForestClassifier

In [52]:
# Rename our feature array
X = all_features

# Create our labels
y = lab_ids.values.ravel()

In [72]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

In [79]:
#scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X) 
X_scaled = scaler.transform(X)

In [80]:
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

In [81]:
#fit neural network
clf.fit(X_scaled, y)

C:\Users\MFarhanMaulana\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [43]:
# instantiate our RF Classifier
rf = RandomForestClassifier(
    n_jobs=4, 
    n_estimators=150,
    class_weight='balanced', # balance classes
    max_depth=3, # shallow tree depth to prevent overfitting
    random_state=0 # set a seed for reproducibility
)

# fit our model
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=150, n_jobs=4, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [71]:
clf.score(X, y)

0.1314883285462653

In [58]:
test_ngram_features = get_ngram_features(test_values, subsequences)
all_test_features = test_ngram_features.join(test_values.drop('sequence', axis=1))

In [59]:
probas = clf.predict_proba(all_test_features)

# Examine first row
probas[0]

array([2.63854372e-05, 3.81260303e-04, 2.50422231e-05, ...,
       1.27402875e-04, 1.01090901e-05, 3.64385097e-05])

In [60]:
submission_format = pd.read_csv(DATA_DIR / 'submission_format.csv', index_col='sequence_id')

In [61]:
submission_format.head()

,00Q4V31T,012VT4JK,028IO5W2,03GRNN7N,03Y3W51H,09MQV1TY,0A4AHRCT,0A9M05NC,0B9GCUVV,0CL7QVG8,...,ZQNGGY33,ZSHS4VJZ,ZT1IP3T6,ZU6860XU,ZU6TVFFU,ZU75P59K,ZUI6TDWV,ZWFD8OHC,ZX06ZDZN,ZZJVE4HO
sequence_id,,,,,,,,,,,,,,,,,,,,,
E0VFT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTRK5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2Z7FZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VJI6E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
721FI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
assert submission_format.shape == probas.shape
assert (clf.classes_ == submission_format.columns).all()

In [65]:
my_submission = pd.DataFrame(data=probas, 
                             columns=clf.classes_, 
                             index=submission_format.index)

In [66]:
my_submission.head()

,00Q4V31T,012VT4JK,028IO5W2,03GRNN7N,03Y3W51H,09MQV1TY,0A4AHRCT,0A9M05NC,0B9GCUVV,0CL7QVG8,...,ZQNGGY33,ZSHS4VJZ,ZT1IP3T6,ZU6860XU,ZU6TVFFU,ZU75P59K,ZUI6TDWV,ZWFD8OHC,ZX06ZDZN,ZZJVE4HO
sequence_id,,,,,,,,,,,,,,,,,,,,,
E0VFT,0.000026,0.000381,0.000025,0.000008,0.004439,0.000030,0.000073,0.000043,0.005851,0.000009,...,0.000026,0.000034,0.000015,0.000012,0.000160,0.003374,0.000025,0.000127,0.000010,0.000036
TTRK5,0.000408,0.000972,0.000401,0.000295,0.002708,0.000447,0.000568,0.000505,0.004145,0.000305,...,0.000417,0.000449,0.000365,0.000336,0.000795,0.001947,0.000411,0.000716,0.000307,0.000576
2Z7FZ,0.000098,0.000743,0.000094,0.000041,0.005177,0.000110,0.000212,0.000145,0.007199,0.000044,...,0.000099,0.000119,0.000065,0.000056,0.000396,0.003857,0.000096,0.000330,0.000048,0.000137
VJI6E,0.000565,0.000697,0.000563,0.000557,0.001150,0.000611,0.000612,0.000634,0.001860,0.000562,...,0.000583,0.000587,0.000604,0.000573,0.000727,0.000811,0.000579,0.000686,0.000546,0.000804
721FI,0.000158,0.000895,0.000153,0.000076,0.004954,0.000176,0.000306,0.000223,0.007057,0.000081,...,0.000160,0.000187,0.000112,0.000099,0.000530,0.003659,0.000155,0.000451,0.000086,0.000221


In [68]:
my_submission.to_csv('submission.csv')